#### Montaje Azure Data Lake para proyecto.

##### Variables

In [0]:
def mount_adls(stge_account_name,container_name):
    client_id=dbutils.secrets.get("db-secret-scope", "Databricks-dev-clientid")
    tenant_id=dbutils.secrets.get("db-secret-scope", "azure-tenant-id")
    client_secret=dbutils.secrets.get("db-secret-scope", "Databricks-dev-secret")

    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    #Desmontar montaje si ya existe
    if any(mount.mountPoint == f"/mnt/{stge_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
          dbutils.fs.unmount(f"/mnt/{stge_account_name}/{container_name}")

    # Optionally, you can add <directory-name> to the source URI of your mount point.
    dbutils.fs.mount(
    source = f"abfss://{container_name}@{stge_account_name}.dfs.core.windows.net/",mount_point = f"/mnt/{stge_account_name}/{container_name}",extra_configs = configs)


In [0]:
storage_account_name ="moviehistorydev"
container_list = ['bronze','silver','gold']

#### Montar contenedor Bronze

In [0]:
for container in container_list:
    mount_adls(storage_account_name, container)

In [0]:
display(dbutils.fs.mounts())

In [0]:
display(dbutils.fs.ls("mnt/moviehistory/demo"))

In [0]:
df_movie = spark.read.format("csv").option("header", "true").load("/mnt/moviehistory/demo/movie.csv")
display(df_movie)

In [0]:
display(dbutils.fs.mounts())

In [0]:
dbutils.fs.unmount("/mnt/moviehistory/demo")